# LINQ - Projection Operators

The `select` clause of a LINQ query *projects* the output sequence. It transforms each input element into the shape of the output sequence.

### Prerequisites

In [ ]:
using System.IO;
using System.Xml.Linq;

public static class InputValues
{       
    private static string customersXml;
    public static string CustomersXml = customersXml ??= File.ReadAllText("_customers.xml");
}

public class Customer
{
    public string CustomerID { get; set; }
    public string CompanyName { get; set; }
    public string Address { get; set; }
    public string City { get; set; }
    public string Region { get; set; }
    public string PostalCode { get; set; }
    public string Country { get; set; }
    public string Phone { get; set; }
    public Order[] Orders { get; set; }
    public override string ToString() => 
        $"{CustomerID} {CompanyName}\n{Address}\n{City}, {Region} {PostalCode} {Country}\n{Phone}";
}

public class Order
{
    public int OrderID { get; set; }
    public DateTime OrderDate { get; set; }
    public decimal Total { get; set; }
    public override string ToString() => $"{OrderID}: {OrderDate:d} for {Total:C2}";
}

public static class Customers
{
    public static List<Customer> CustomerList { get; } =
        (from e in XDocument.Parse(InputValues.CustomersXml).Root.Elements("customer")
            select new Customer
            {
                CustomerID = (string)e.Element("id"),
                CompanyName = (string)e.Element("name"),
                Address = (string)e.Element("address"),
                City = (string)e.Element("city"),
                Region = (string)e.Element("region"),
                PostalCode = (string)e.Element("postalcode"),
                Country = (string)e.Element("country"),
                Phone = (string)e.Element("phone"),
                Orders = (
                from o in e.Elements("orders").Elements("order")
                select new Order
                {
                    OrderID = (int)o.Element("id"),
                    OrderDate = (DateTime)o.Element("orderdate"),
                    Total = (decimal)o.Element("total")
                }).ToArray()
            }).ToList();
}

List<Customer> GetCustomerList() => Customers.CustomerList;

## Compound select with where and assignment

This sample uses a compound `from` clause to select all orders where the order total is greater than 2000.00 and uses `from` assignment to avoid requesting the total twice.

In [ ]:
List<Customer> customers = GetCustomerList();

var orders = from c in customers
             from o in c.Orders
             where o.Total >= 2000.0M
             select (c.CustomerID, o.OrderID, o.Total);

foreach (var order in orders)
{
    Console.WriteLine($"Customer: {order.CustomerID}, Order: {order.OrderID}, Total value: {order.Total}");
}

## Compound select with multiple where clauses

This sample uses multiple `from` clauses so that filtering on customers can be done before selecting their orders. This makes the query more efficient by not selecting and then discarding orders for customers outside of Washington.

In [ ]:
List<Customer> customers = GetCustomerList();

DateTime cutoffDate = new DateTime(1997, 1, 1);

var orders = from c in customers
             where c.Region == "WA"
             from o in c.Orders
             where o.OrderDate >= cutoffDate
             select (c.CustomerID, o.OrderID);

foreach (var order in orders)
{
    Console.WriteLine($"Customer: {order.CustomerID}, Order: {order.OrderID}");
}

## Compound select with index

This sample uses an indexed `SelectMany` clause to select all orders, while referring to customers by the order in which they are returned from the query.

In [ ]:
List<Customer> customers = GetCustomerList();

var customerOrders =
    customers.SelectMany(
        (cust, custIndex) =>
        cust.Orders.Select(o => "Customer #" + (custIndex + 1) +
                                " has an order with OrderID " + o.OrderID));

foreach (var order in customerOrders)
{
    Console.WriteLine(order);
}

**Next: [Partition operators  &raquo;](./partitions.ipynb) Previous: [Restrictions from multiple inputs &laquo;](./projections-4.ipynb)**

**[Home](../index.ipynb)**